In [1]:
#pip install tensorflow

In [2]:
import os
import numpy as np
import cv2
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [3]:
directory = "TImages/"

In [4]:
image_size = (244, 244)
classes = ["0", "1", "2"]

In [5]:
# Create an ImageDataGenerator instance with data augmentation settings
datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest',
    samplewise_center=True,
    samplewise_std_normalization=True,
)

In [6]:
training_data = []

def create_training_data():
    for category in classes:
        path = os.path.join(directory, category)
        class_num = classes.index(category)
        for img in os.listdir(path):
            try:
                img_array = cv2.imread(os.path.join(path, img))
                new_array = cv2.resize(img_array, image_size)

                # Generate and store augmented images
                augmented_images = []
                augmented_images.append(new_array)  # Original image
                img_array_aug = new_array.reshape((1,) + new_array.shape)
                i = 0
                for batch in datagen.flow(img_array_aug, batch_size=1):
                    augmented_images.append(batch[0])
                    i += 1
                    if i >= 5:  # Generate 5 augmented images per input image
                        break

                for augmented_image in augmented_images:
                    image_hsv = cv2.cvtColor(augmented_image, cv2.COLOR_BGR2HSV)
                    training_data.append([image_hsv, class_num])

            except Exception as e:
                pass

In [7]:
create_training_data()

In [8]:
lenofimage = len(training_data)
print(lenofimage)

5850


In [9]:
X=[]
y=[]

for categories, label in training_data:
    X.append(categories)
    y.append(label)
X= np.array(X).reshape(lenofimage,-1)

In [10]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, )

In [12]:
## GB MODEL

In [13]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [14]:
# Feature Scaling
sc = StandardScaler()

X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [19]:
# Hyperparameter Tuning using GridSearchCV
param_grid = {'n_estimators': [100, 150],
              'learning_rate': [0.1, 0.3],
              'max_depth': [5, 8],
              'max_features': ['sqrt', 'log2'],
              'loss': ['log_loss'],
              'subsample': [0.1, 0.5],
             
             }

In [20]:
gb = GradientBoostingClassifier()

In [21]:
# Create the GridSearch object without cross-validation
grid_search = GridSearchCV(gb, param_grid, cv=5, verbose=2,error_score='raise')  # Set cv=None for no cross-validation

In [22]:
# Fit the GridSearch object to your data
grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 32 candidates, totalling 160 fits
[CV] END learning_rate=0.1, loss=log_loss, max_depth=5, max_features=sqrt, n_estimators=100, subsample=0.1; total time=  27.3s
[CV] END learning_rate=0.1, loss=log_loss, max_depth=5, max_features=sqrt, n_estimators=100, subsample=0.1; total time=  27.1s
[CV] END learning_rate=0.1, loss=log_loss, max_depth=5, max_features=sqrt, n_estimators=100, subsample=0.1; total time=  27.3s
[CV] END learning_rate=0.1, loss=log_loss, max_depth=5, max_features=sqrt, n_estimators=100, subsample=0.1; total time=  26.9s
[CV] END learning_rate=0.1, loss=log_loss, max_depth=5, max_features=sqrt, n_estimators=100, subsample=0.1; total time=  27.0s
[CV] END learning_rate=0.1, loss=log_loss, max_depth=5, max_features=sqrt, n_estimators=100, subsample=0.5; total time= 3.7min
[CV] END learning_rate=0.1, loss=log_loss, max_depth=5, max_features=sqrt, n_estimators=100, subsample=0.5; total time= 6.3min
[CV] END learning_rate=0.1, loss=log_loss, max_de

[CV] END learning_rate=0.1, loss=log_loss, max_depth=8, max_features=log2, n_estimators=100, subsample=0.5; total time=  11.5s
[CV] END learning_rate=0.1, loss=log_loss, max_depth=8, max_features=log2, n_estimators=100, subsample=0.5; total time=  11.2s
[CV] END learning_rate=0.1, loss=log_loss, max_depth=8, max_features=log2, n_estimators=100, subsample=0.5; total time=  11.6s
[CV] END learning_rate=0.1, loss=log_loss, max_depth=8, max_features=log2, n_estimators=100, subsample=0.5; total time=  11.3s
[CV] END learning_rate=0.1, loss=log_loss, max_depth=8, max_features=log2, n_estimators=100, subsample=0.5; total time=  11.5s
[CV] END learning_rate=0.1, loss=log_loss, max_depth=8, max_features=log2, n_estimators=150, subsample=0.1; total time=   5.9s
[CV] END learning_rate=0.1, loss=log_loss, max_depth=8, max_features=log2, n_estimators=150, subsample=0.1; total time=   5.7s
[CV] END learning_rate=0.1, loss=log_loss, max_depth=8, max_features=log2, n_estimators=150, subsample=0.1; tot

[CV] END learning_rate=0.3, loss=log_loss, max_depth=8, max_features=sqrt, n_estimators=150, subsample=0.1; total time=  59.4s
[CV] END learning_rate=0.3, loss=log_loss, max_depth=8, max_features=sqrt, n_estimators=150, subsample=0.1; total time=  56.6s
[CV] END learning_rate=0.3, loss=log_loss, max_depth=8, max_features=sqrt, n_estimators=150, subsample=0.1; total time=  59.5s
[CV] END learning_rate=0.3, loss=log_loss, max_depth=8, max_features=sqrt, n_estimators=150, subsample=0.1; total time= 1.3min
[CV] END learning_rate=0.3, loss=log_loss, max_depth=8, max_features=sqrt, n_estimators=150, subsample=0.1; total time= 1.1min
[CV] END learning_rate=0.3, loss=log_loss, max_depth=8, max_features=sqrt, n_estimators=150, subsample=0.5; total time= 5.2min
[CV] END learning_rate=0.3, loss=log_loss, max_depth=8, max_features=sqrt, n_estimators=150, subsample=0.5; total time= 5.0min
[CV] END learning_rate=0.3, loss=log_loss, max_depth=8, max_features=sqrt, n_estimators=150, subsample=0.5; tot

GridSearchCV(cv=5, error_score='raise', estimator=GradientBoostingClassifier(),
             param_grid={'learning_rate': [0.1, 0.3], 'loss': ['log_loss'],
                         'max_depth': [5, 8], 'max_features': ['sqrt', 'log2'],
                         'n_estimators': [100, 150], 'subsample': [0.1, 0.5]},
             verbose=2)

In [23]:
# Get the best estimator and evaluate it
best_nb = grid_search.best_estimator_
y_test_pred = best_nb.predict(X_test)
accuracy = accuracy_score(y_test, y_test_pred)
print("Best Gaussian Naive Bayes Parameters:", grid_search.best_params_)
print("Accuracy: {:.2f}%".format(accuracy * 100))

Best Gaussian Naive Bayes Parameters: {'learning_rate': 0.1, 'loss': 'log_loss', 'max_depth': 5, 'max_features': 'log2', 'n_estimators': 100, 'subsample': 0.5}
Accuracy: 95.04%


In [24]:
# work progressing...........


In [25]:
grid_search.cv_results_

{'mean_fit_time': array([ 26.79219799, 341.68242688,  97.90530262, 297.85650697,
          8.43672547,  12.14344125,   6.29513726,  14.62389903,
         57.13575745, 205.39720654,  59.04720416, 300.9145391 ,
          4.09094062,  11.09113092,   5.47524753,  16.1162941 ,
         26.19519186, 126.75956292,  38.56109529, 189.29962201,
          3.32509069,   7.54397707,   4.30650554,  10.65969262,
         37.77285595, 202.37697239,  63.57156696, 306.19361453,
          4.18208065,  11.65900888,   5.55029669,  16.31768837]),
 'std_fit_time': array([1.52260807e-01, 6.13758276e+01, 3.49388464e-01, 1.01104252e+01,
        1.68185839e-01, 2.83225796e+00, 1.98809431e-01, 1.71543960e+00,
        4.63701765e+00, 9.28912484e+00, 1.14721890e+00, 7.84657994e-01,
        1.81302488e-02, 1.37434722e-01, 8.64966087e-02, 3.47608725e-01,
        5.27705047e-02, 2.86272502e-01, 1.19307373e-01, 4.86796005e-01,
        3.75037236e-02, 2.86449067e-02, 7.83057234e-02, 3.87814367e-02,
        6.40844975e-0

In [26]:
df = pd.DataFrame(grid_search.cv_results_)
df

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_learning_rate,param_loss,param_max_depth,param_max_features,param_n_estimators,param_subsample,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,26.792198,0.152261,0.399800,1.937914e-03,0.1,log_loss,5,sqrt,100,0.1,"{'learning_rate': 0.1, 'loss': 'log_loss', 'ma...",0.860043,0.850427,0.034188,0.054487,0.880342,0.535897,0.401524,19
1,341.682427,61.375828,1.740069,7.816380e-02,0.1,log_loss,5,sqrt,100,0.5,"{'learning_rate': 0.1, 'loss': 'log_loss', 'ma...",0.957265,0.955128,0.948718,0.956197,0.960470,0.955556,0.003858,7
2,97.905303,0.349388,1.661068,4.295769e-01,0.1,log_loss,5,sqrt,150,0.1,"{'learning_rate': 0.1, 'loss': 'log_loss', 'ma...",0.917735,0.030983,0.158120,0.034188,0.822650,0.392735,0.393679,25
3,297.856507,10.110425,0.950420,1.817319e-01,0.1,log_loss,5,sqrt,150,0.5,"{'learning_rate': 0.1, 'loss': 'log_loss', 'ma...",0.959402,0.956197,0.944444,0.956197,0.963675,0.955983,0.006389,6
4,8.436725,0.168186,1.218867,5.232594e-02,0.1,log_loss,5,log2,100,0.1,"{'learning_rate': 0.1, 'loss': 'log_loss', 'ma...",0.239316,0.708333,0.896368,0.210470,0.761752,0.563248,0.283128,17
5,12.143441,2.832258,0.850249,1.819823e-01,0.1,log_loss,5,log2,100,0.5,"{'learning_rate': 0.1, 'loss': 'log_loss', 'ma...",0.958333,0.962607,0.957265,0.961538,0.962607,0.960470,0.002241,1
6,6.295137,0.198809,0.725229,8.012507e-02,0.1,log_loss,5,log2,150,0.1,"{'learning_rate': 0.1, 'loss': 'log_loss', 'ma...",0.797009,0.185897,0.108974,0.309829,0.356838,0.351709,0.239368,28
7,14.623899,1.715440,0.865900,1.862697e-01,0.1,log_loss,5,log2,150,0.5,"{'learning_rate': 0.1, 'loss': 'log_loss', 'ma...",0.961538,0.960470,0.955128,0.959402,0.962607,0.959829,0.002582,3
8,57.135757,4.637018,1.059934,2.052164e-01,0.1,log_loss,8,sqrt,100,0.1,"{'learning_rate': 0.1, 'loss': 'log_loss', 'ma...",0.180556,0.896368,0.042735,0.858974,0.862179,0.568162,0.375513,16
9,205.397207,9.289125,0.409405,6.251169e-03,0.1,log_loss,8,sqrt,100,0.5,"{'learning_rate': 0.1, 'loss': 'log_loss', 'ma...",0.946581,0.961538,0.951923,0.954060,0.960470,0.954915,0.005547,8


In [27]:
df[[ 'params', 'mean_test_score',  'rank_test_score']]

,params,mean_test_score,rank_test_score
0,"{'learning_rate': 0.1, 'loss': 'log_loss', 'ma...",0.535897,19
1,"{'learning_rate': 0.1, 'loss': 'log_loss', 'ma...",0.955556,7
2,"{'learning_rate': 0.1, 'loss': 'log_loss', 'ma...",0.392735,25
3,"{'learning_rate': 0.1, 'loss': 'log_loss', 'ma...",0.955983,6
4,"{'learning_rate': 0.1, 'loss': 'log_loss', 'ma...",0.563248,17
5,"{'learning_rate': 0.1, 'loss': 'log_loss', 'ma...",0.960470,1
6,"{'learning_rate': 0.1, 'loss': 'log_loss', 'ma...",0.351709,28
7,"{'learning_rate': 0.1, 'loss': 'log_loss', 'ma...",0.959829,3
8,"{'learning_rate': 0.1, 'loss': 'log_loss', 'ma...",0.568162,16
9,"{'learning_rate': 0.1, 'loss': 'log_loss', 'ma...",0.954915,8


In [28]:
grid_search.best_score_

0.9604700854700855

In [29]:
grid_search.best_params_

{'learning_rate': 0.1,
 'loss': 'log_loss',
 'max_depth': 5,
 'max_features': 'log2',
 'n_estimators': 100,
 'subsample': 0.5}